# Getting Started with the Agent Development Kit (ADK) with Neo4j

This notebook shows how to use the Vertex AI Agent Framework to setup a multi-agent system integrating Neo4j.

The Vertex AI ADK is a python-based SDK that empowers developers to build multi-agent applications with custom logic, tools, and integrations.

In this notebook you will learn how to do the following:

*  Install Agent Development Kit
*  Define functions as tools
*  Define a database and analyst agent
*  Create a "Bom & Supplier Research" Multi-Agent


## Learn more about Neo4j

* https://neo4j.com/genai
* https://neo4j.com/developer
* https://graphrag.com
* https://neo4j.com/labs/genai-ecosystem
* [Neo4j and MCP Toolbox](https://neo4j.com/blog/developer/ai-agents-gen-ai-toolbox/)

## Learn more about the ADK

* [Launch Blog](https://cloud.google.com/blog/products/ai-machine-learning/build-and-manage-multi-system-agents-with-vertex-ai)
* [Developer Blog](https://developers.googleblog.com/en/agent-development-kit-easy-to-build-multi-agent-applications/)
* https://pypi.org/project/google-adk/
* https://google.github.io/adk-docs/
* https://github.com/google/adk-samples/
* [LLM Agents](https://google.github.io/adk-docs/agents/llm-agents/#putting-it-together-example)
* [Deploy to Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/)

# Install the Agent Development Kit

In [1]:
# from google.colab import auth
# auth.authenticate_user()
!gcloud auth application-default login > /dev/null 2>&1

In [6]:
%pip install --upgrade google-cloud-aiplatform[adk,agent_engines] neo4j-rust-ext python-dotenv

Note: you may need to restart the kernel to use updated packages.


# Developer API or Vertex AI

The ADK supports two APIs:

* Google Gemini API (AI Studio)
* Vertex AI Gemini API.

In [1]:
from dotenv import load_dotenv
import getpass
import os

load_dotenv('nb.env', override=True)

if not os.environ.get('NEO4J_URI'):
    os.environ['NEO4J_URI'] = getpass.getpass('NEO4J_URI:\n')
if not os.environ.get('NEO4J_USERNAME'):
    os.environ['NEO4J_USERNAME'] = getpass.getpass('NEO4J_USERNAME:\n')
if not os.environ.get('NEO4J_PASSWORD'):
    os.environ['NEO4J_PASSWORD'] = getpass.getpass('NEO4J_PASSWORD:\n')

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

In [2]:
# Only run this block for ML Developer API. Use your own API key.
import os

GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "0"


In [9]:
# Only run this block for Vertex AI API Use your own project / location.
# import os
#
# GOOGLE_CLOUD_PROJECT = "neo4jeventdemos" #@param {type:"string"}
#
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
# os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT
# os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"

# Neo4j Database Connectivity

We define a small class `neo4jDatabase` that manages connecting to Neo4j and running read queries.

In [3]:
import logging

logger = logging.getLogger('agent_neo4j_cypher')
logger.info("Initializing Database for tools")

import logging

logging.getLogger("neo4j").setLevel(logging.ERROR)
logging.getLogger("google_genai").setLevel(logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="google_genai.types")
warnings.filterwarnings("ignore", category=UserWarning, module="neo4j.notifications")


In [4]:
from neo4j import GraphDatabase
from typing import Any, List, Dict
import re

class neo4jDatabase:
    def __init__(self,  neo4j_uri: str, neo4j_username: str, neo4j_password: str):
        """Initialize connection to the Neo4j database"""
        logger.debug(f"Initializing database connection to {neo4j_uri}")
        d = GraphDatabase.driver(neo4j_uri, auth=(neo4j_username, neo4j_password))
        d.verify_connectivity()
        self.driver = d

    def is_write_query(self, query: str) -> bool:
      return re.search(r"\b(MERGE|CREATE|SET|DELETE|REMOVE|ADD)\b", query, re.IGNORECASE) is not None

    #TODO: Use result transformer here to r: r.data()
    def _execute_query(self, query: str, params: Dict[str, Any] | None = None) -> List[Dict[str, Any]]:
        """Execute a Cypher query and return results as a list of dictionaries"""
        logger.debug(f"Executing query: {query}")
        try:
            if self.is_write_query(query):
                logger.error(f"Write query not supported {query}")
                raise "Write Queries are not supported in this agent"
                # logger.debug(f"Write query affected {counters}")
                # result = self.driver.execute_query(query, params)
                # counters = vars(result.summary.counters)
                # return [counters]
            else:
                #TODO: Add Routing here for effciency
                result = self.driver.execute_query(query, params)
                results = [dict(r) for r in result.records]
                logger.debug(f"Read query returned {len(results)} rows")
                return results
        except Exception as e:
            logger.error(f"Database error executing query: {e}\n{query}")
            raise

In [5]:
db = neo4jDatabase(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)

In [6]:
# Testing database connection
db._execute_query("RETURN 1")

[{'1': 1}]

## Defining Functions for our Database Agent

* get_schema - Retrieve Database Schema for the LLM
* execute_read_query - Execute Cypher Read Query

In [7]:
def get_schema() -> List[Dict[str,Any]]:
  """Get the schema of the database, returns node-types(labels) with their types and attributes and relationships between node-labels
  Args: None
  Returns:
    list[dict[str,Any]]: A list of dictionaries representing the schema of the database
    For example
    ```
    [{'label': 'Person','attributes': {'summary': 'STRING','id': 'STRING unique indexed', 'name': 'STRING indexed'},
      'relationships': {'HAS_PARENT': 'Person', 'HAS_CHILD': 'Person'}}]
    ```
  """
  try:
      results = db._execute_query(
              """
call apoc.meta.data() yield label, property, type, other, unique, index, elementType
where elementType = 'node' and not label starts with '_'
with label,
collect(case when type <> 'RELATIONSHIP' then [property, type + case when unique then " unique" else "" end + case when index then " indexed" else "" end] end) as attributes,
collect(case when type = 'RELATIONSHIP' then [property, head(other)] end) as relationships
RETURN label, apoc.map.fromPairs(attributes) as attributes, apoc.map.fromPairs(relationships) as relationships
              """
          )
      return results
  except Exception as e:
      return [{"error":str(e)}]

In [8]:
get_schema()

[{'label': 'Supplier',
  'attributes': {'annual_spend': 'FLOAT',
   'tier': 'STRING',
   'sub_type': 'STRING',
   'code': 'STRING indexed'},
  'relationships': {'LOCATED_AT': 'GeoLocation'}},
 {'label': 'Item',
  'attributes': {'desc_embedding': 'LIST',
   'description': 'STRING',
   'family': 'STRING',
   'is_finished_product': 'BOOLEAN',
   'name': 'STRING',
   'sku_id': 'STRING indexed'},
  'relationships': {'AT': 'Supplier', 'BOM': 'Item', 'DEPENDS_ON': 'Country'}},
 {'label': 'Customer',
  'attributes': {'annual_revenue': 'FLOAT', 'code': 'INTEGER indexed'},
  'relationships': {'LOCATED_AT': 'GeoLocation'}},
 {'label': 'GeoLocation',
  'attributes': {'longitude': 'FLOAT',
   'latitude': 'FLOAT',
   'geo_point': 'POINT indexed'},
  'relationships': {}},
 {'label': 'Country',
  'attributes': {'name': 'STRING', 'code': 'STRING indexed'},
  'relationships': {}}]

In [9]:
from typing import Optional


def execute_read_query(query: str, params: Optional[Dict[str, Any]]=None) -> List[Dict[str, Any]]:
    """
    Execute a Neo4j Cypher query and return results as a list of dictionaries
    Args:
        query (str): The Cypher query to execute
        params (dict[str, Any], optional): The parameters to pass to the query or None.
    Raises:
        Exception: If there is an error executing the query
    Returns:
        list[dict[str, Any]]: A list of dictionaries representing the query results
    """
    try:
        if params is None:
            params = {}
        results = db._execute_query(query, params)
        return results
    except Exception as e:
        return [{"error":str(e)}]

In [10]:
execute_read_query("RETURN 1", None)

[{'1': 1}]

## Defining Functions for our Investment Research Agent

* get_schema - Retrieve Database Schema for the LLM
* get_investors - Returns the investor in the company with this name or id.

In [42]:
def get_finished_product_families() -> List[Dict[str, Any]]:
    """
    Get the different type of Product Families - for finished products only.
    Use this when asked for different types, families, or groups of finished products.
    """
    try:
        results = db._execute_query("""
        MATCH (i:Item)
        WHERE i.is_finished_product
        RETURN i.family AS family, count(*) AS numberOfProducts
        """)
        return results
    except Exception as e:
        return [{"error":str(e)}]
#test
get_finished_product_families()

[{'family': 'FarmTractor', 'numberOfProducts': 21521},
 {'family': 'SeedPlanter', 'numberOfProducts': 21488},
 {'family': 'FieldSprayer', 'numberOfProducts': 21519},
 {'family': 'CropHarvester', 'numberOfProducts': 21406},
 {'family': 'HayCollector', 'numberOfProducts': 21671},
 {'family': 'CabWiringUnit', 'numberOfProducts': 17},
 {'family': 'ControlAssembly', 'numberOfProducts': 9},
 {'family': 'MachineRig', 'numberOfProducts': 14},
 {'family': 'HydraulicSystem', 'numberOfProducts': 3},
 {'family': 'ChassisFrame', 'numberOfProducts': 9},
 {'family': 'DrivePlatform', 'numberOfProducts': 10}]

In [43]:
def get_component_families() -> List[Dict[str, Any]]:
    """
    Get the different type of Component Families - for components only -not finished products.
    Use this when asked for different types, families or groups of components and/or parts.
    """
    try:
        results = db._execute_query("""
        MATCH (i:Item)
        WHERE NOT i.is_finished_product
        RETURN i.family AS family, count(*) AS numberOfComponents
        """)
        return results
    except Exception as e:
        return [{"error":str(e)}]
#test
get_component_families()

[{'family': 'MachineRig', 'numberOfComponents': 20017},
 {'family': 'DrivePlatform', 'numberOfComponents': 19896},
 {'family': 'HydraulicSystem', 'numberOfComponents': 20192},
 {'family': 'ChassisFrame', 'numberOfComponents': 19893},
 {'family': 'ControlAssembly', 'numberOfComponents': 19824},
 {'family': 'CabWiringUnit', 'numberOfComponents': 19886},
 {'family': 'Bushing', 'numberOfComponents': 162},
 {'family': 'Spring', 'numberOfComponents': 144},
 {'family': 'GearSet', 'numberOfComponents': 169},
 {'family': 'ElectricalControlBox', 'numberOfComponents': 77},
 {'family': 'ComponentModule', 'numberOfComponents': 63},
 {'family': 'Board', 'numberOfComponents': 82},
 {'family': 'SteeringColumn', 'numberOfComponents': 65},
 {'family': 'FrameSegment', 'numberOfComponents': 62},
 {'family': 'SensorModule', 'numberOfComponents': 67},
 {'family': 'OperatorCab', 'numberOfComponents': 12},
 {'family': 'AxleAssembly', 'numberOfComponents': 79},
 {'family': 'ControlUnit', 'numberOfComponents': 

In [44]:
def get_item(sku_id: str) -> dict[str, Any]:
    """
    Gets Item properties
    Args:
        sku_id (str): code uniquely identifying the item which can be a component or finished product
    Returns:
        dict[str, Any]: item properties
    """
    try:
        results = db._execute_query("""
        MATCH (i:Item {sku_id: "35472245A"})
        RETURN i.sku_id AS sku_id, i.name AS name, i.family AS family, i.is_finished_product AS is_finished_product
        """, {"sku_id": sku_id})
        return results
    except Exception as e:
        return {"error": str(e)}

In [45]:
get_item("35472245A")

[{'sku_id': '35472245A',
  'name': 'FarmTractor_7YUC0',
  'family': 'FarmTractor',
  'is_finished_product': True}]

In [46]:
def get_known_item_country_dependencies(sku_id: str) -> List[Dict[str, Any]]:
    """
    Retrieve all known country dependencies for a given item, which can represent either a finished product, a component, or both. This includes data on sourcing, manufacturing, or other relationships tied to specific countries.
    Dependencies are determined recursively by examining all connected components in the item's bill of materials (BOM). For each component, this function identifies:
    1. **Known Country Dependencies:** A list of countries and their corresponding dependency descriptions (e.g., sourcing steel, manufacturing).
    2. **Supply Chain Dependencies:** A visual representation of how components with such country dependencies are linked to the provided item through potentially multiple stages of the BOM hierarchy, represented as a sequence of components connected by arrows, culminating in the provided item.

    Note: This function only retrieves available data, which may be incomplete due to limited internal intelligence.

    Args:
        sku_id (str): code uniquely identifying the component or finished product
    Returns:
        list[dict[str, Any]]: a list of components in this items bill of materials with country dependencies along with their country dependencies and how they relate to the item in the bill of materials hierarchy.
    """
    try:
        results = db._execute_query("""
        MATCH (i:Item {sku_id: $sku_id})
        MATCH path=(i)<-[:BOM*]-(comp:Item)-[r:DEPENDS_ON]->(c:Country)
        RETURN
        comp.sku_id AS sku_id,
        comp.name AS name,
        collect({country: c.name, dependency: r.description}) AS known_country_dependencies,
        ' <- ' + apoc.text.join([n IN tail(nodes(path))[..-1] | n.sku_id + ' (' + coalesce(n.name, '') + ')' ], ' <- ') AS supply_chain_dependency
        """, {"sku_id": sku_id})
        return results
    except Exception as e:
        return [{"error": str(e)}]

In [47]:
get_known_item_country_dependencies("35472245A")

[{'sku_id': 'M7307U91X',
  'name': 'RawWire_YCBEO',
  'known_country_dependencies': [{'dependency': 'sourcing raw copper cathode',
    'country': 'Chile'},
   {'dependency': 'drawing and annealing facilities', 'country': 'Mexico'},
   {'dependency': 'drawing and annealing facilities', 'country': 'Viet Nam'},
   {'dependency': 'sourcing raw copper cathode', 'country': 'Peru'}],
  'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35523080X (CabWiringUnit_UJ3OE) <- M7307U91X (RawWire_YCBEO)'},
 {'sku_id': 'M7307U92X',
  'name': 'PrecisionBolt_O12J1',
  'known_country_dependencies': [{'dependency': 'final machining and threading processes',
    'country': 'United States of America'},
   {'dependency': 'sourcing raw titanium sponge', 'country': 'Japan'},
   {'dependency': 'procuring alloying elements (aluminum and vanadium)',
    'country': 'Russian Federation'}],
  'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35523087X (CabWiringUnit_CZ0N2) <- M7307U92X

In [48]:
def get_item_bill_of_materials(sku_id: str) -> List[Dict[str, Any]]:
    """
    Retrieve the complete Bill of Materials (BOM) hierarchy for a given finished product or component identified by the SKU.
    This function provides a detailed view of all components involved in the production of the specified item.
    It identifies components recursively linked to the given item through the BOM hierarchy.

    Key Features:
    1. **Bill of Material Dependencies:** Identifies all components recursively linked to the given item through the BOM hierarchy.
    2. **Dependency Path Representation:** Produces a flattened tree structure where each entry represents a sequence of components
       connected by arrows, showing the hierarchical relationships from the base components to the provided item.


    Args:
        sku_id (str): The SKU (Stock Keeping Unit) that uniquely identifies a product, component, or part.

    Returns:
        list[dict[str, Any]]: A list of dependency paths represented as strings. Each path shows the components in the BOM hierarchy
        leading to the specified item, connected by arrows (` <- `). For example:
            " <- 35472245 (CabWiringUnit_Z5OO3)"
            " <- 35472245 (CabWiringUnit_Z5OO3) <- 35475154X (CabWiringUnit_VT75O)"
            " <- 35472245 (CabWiringUnit_Z5OO3) <- 35475154X (CabWiringUnit_VT75O) <- M3571002BX (CabWiringUnit_A9FH4)"
    """

    try:
        results = db._execute_query("""
        MATCH (i:Item {sku_id: $sku_id})
        MATCH path=(i)<-[:BOM*]-(:Item)
        RETURN ' <- ' + apoc.text.join([n IN tail(nodes(path)) | n.sku_id + ' (' + coalesce(n.name, '') + ')' ], ' <- ') AS supply_chain_dependency
        """, {"sku_id": sku_id})
        return results
    except Exception as e:
        return [{"error": str(e)}]


In [49]:
get_item_bill_of_materials("35472245A")

[{'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6495003X (DrivePlatform_SBK8U)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6499003X (MachineRig_RFYOE)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6499002X (HydraulicSystem_EE22P)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35370826 (MachineRig_INWWQ)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35433612 (ChassisFrame_2GN2W)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35475030 (ElectricalControlBox_6ROWJ)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 28028534X (ControlAssembly_JRO9M)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35475154X (CabWiringUnit_VT75O)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35475154X (CabWiringUnit_VT75O) <- M357

In [58]:
def get_standard_country_codes_and_names() -> List[Dict[str, Any]]:
    """
    Get the standardized country codes and names that are mentioned in the BOM and supply chain data.
    Other methods that have country names and code as input will only match data if with these standardized codes/names.
    Note that not all countries may be listed, as not all countries are mentioned in intel.
    """
    try:
        results = db._execute_query("""
        MATCH (c:Country)
        RETURN c.code as code, c.name as name
        """)
        return results
    except Exception as e:
        return [{"error":str(e)}]
#test
get_standard_country_codes_and_names()

[{'CountryCode': 'AUS', 'CountryName': 'Australia'},
 {'CountryCode': 'BEL', 'CountryName': 'Belgium'},
 {'CountryCode': 'BRA', 'CountryName': 'Brazil'},
 {'CountryCode': 'CAN', 'CountryName': 'Canada'},
 {'CountryCode': 'CHE', 'CountryName': 'Switzerland'},
 {'CountryCode': 'CHL', 'CountryName': 'Chile'},
 {'CountryCode': 'CHN', 'CountryName': 'China'},
 {'CountryCode': 'COL', 'CountryName': 'Colombia'},
 {'CountryCode': 'DEU', 'CountryName': 'Germany'},
 {'CountryCode': 'EGY', 'CountryName': 'Egypt'},
 {'CountryCode': 'GIN', 'CountryName': 'Guinea'},
 {'CountryCode': 'IDN', 'CountryName': 'Indonesia'},
 {'CountryCode': 'IND', 'CountryName': 'India'},
 {'CountryCode': 'IRL', 'CountryName': 'Ireland'},
 {'CountryCode': 'ISL', 'CountryName': 'Iceland'},
 {'CountryCode': 'ITA', 'CountryName': 'Italy'},
 {'CountryCode': 'JPN', 'CountryName': 'Japan'},
 {'CountryCode': 'KAZ', 'CountryName': 'Kazakhstan'},
 {'CountryCode': 'KOR', 'CountryName': 'Korea, Republic of'},
 {'CountryCode': 'MAR',

In [63]:
from typing import List, Dict


def get_products_with_country_dependencies(country_code:str, dependency_search_term: Optional[str] = None) -> List[Dict[str, Any]]:
    """
    Identify finished products that are affected by dependencies on a specific material or resource from a given country.

    This function traces the supply chain of finished products to specific materials or resources sourced from a particular country. By analyzing the Bill of Materials (BOM) hierarchy,
    it identifies how dependencies on components are linked to the production of finished products.

    Key Features:
    1. **Country-Specific Dependencies:** Finds components linked to a specific country and determines how their dependencies
       impact finished products.
    2. **Dependency Search:** Allows optional filtering of dependencies by a search term (e.g., "nickel").
    3. **Supply Chain Traceability:** Provides detailed paths showing how the affected components feed into finished products.

    Args:
        country_code (str): The standardized ISO 3166 alpha-3 country code (e.g., "CAN" for Canada) to trace dependencies related to a specific country.
        dependency_search_term (Optional[str]): A search term to filter specific dependencies (e.g., "nickel").
            If omitted, all dependencies for the country are considered. keep the search terms short and simple - exact matching is used for criteria

    Returns:
        List[Dict[str, Any]]: A list where each element represents a finished product impacted by the specified dependencies.
        Each entry includes:
            - **product_sku_id (str):** The SKU of the finished product.
            - **product_name (str):** The name of the finished product.
            - **dependencies (list):** A list of dictionaries, with each dictionary representing:
                - **depends_on_item_sku_id (str):** The SKU of the component causing the dependency.
                - **depends_on_item_name (str):** The name of the component causing the dependency.
                - **country_dependency_of_item (list[dict]):** A list of country dependencies associated with the component,
                  including:
                    - **dependency (str):** A description of the dependency (e.g., "sourcing nickel").
                    - **country (str):** The country associated with the dependency (e.g., "Canada").
                - **supply_chain_dependency_for_product (str):** A trace of the supply chain relationship between the component
                  and the finished product. This trace is represented as a sequence of SKUs and component names connected by arrows (e.g.,
                  " <- D0301002 (PrecisionBolt_252A1) <- 13952372 (ChassisFrame_3AC3Z)").

    Examples:
        - **Query:** country_code = "CAN", dependency_search_term = "nickel"
        - **Output:**
            [
                {
                    "product_sku_id": "35513559",
                    "product_name": "FarmTractor_VLM4F",
                    "dependencies": [
                        {
                            "depends_on_item_sku_id": "D0301002",
                            "depends_on_item_name": "PrecisionBolt_252A1",
                            "country_dependency_of_item": [
                                {
                                    "dependency": "sourcing nickel",
                                    "country": "Canada"
                                }
                            ],
                            "supply_chain_dependency_for_product": " <- D0301002 (PrecisionBolt_252A1) <- 13978746 (CabWiringUnit_EVKCK)"
                        }
                    ]
                },
                ...
            ]
    """

    try:
        if dependency_search_term is None:
            results = db._execute_query("""
            MATCH (i:Item)-[r:DEPENDS_ON]->(c:Country {code:$countryCode})
            WITH i,r,c
            MATCH path = (i)-[:BOM*]->(p {is_finished_product:true})
            WITH p.sku_id AS product_sku_id, p.name AS product_name,  i.sku_id AS depends_on_item_sku_id, i.name AS depends_on_item_name, collect({country: c.name, dependency: r.description}) AS known_country_dependency,
                ' <- ' + apoc.text.join([n IN reverse(nodes(path)[..-1]) | n.sku_id + ' (' + coalesce(n.name, '') + ')' ], ' <- ') AS supply_chain_dependency
            RETURN  product_sku_id, product_name, collect({depends_on_item_sku_id: depends_on_item_sku_id, depends_on_item_name: depends_on_item_name, country_dependency_of_item: known_country_dependency, supply_chain_dependency_for_product: supply_chain_dependency}) AS dependencies
            """, {"countryCode": country_code, "dependencySearchTerm": dependency_search_term})
        else:
            results = db._execute_query("""
            MATCH (i:Item)-[r:DEPENDS_ON]->(c:Country {code:$countryCode})
            WHERE  lower(r.description) CONTAINS lower($dependencySearchTerm)
            WITH i,r,c
            MATCH path = (i)-[:BOM*]->(p {is_finished_product:true})
            WITH p.sku_id AS product_sku_id, p.name AS product_name,  i.sku_id AS depends_on_item_sku_id, i.name AS depends_on_item_name, collect({country: c.name, dependency: r.description}) AS known_country_dependency,
                ' <- ' + apoc.text.join([n IN reverse(nodes(path)[..-1])| n.sku_id + ' (' + coalesce(n.name, '') + ')' ], ' <- ') AS supply_chain_dependency
            RETURN  product_sku_id, product_name, collect({depends_on_item_sku_id: depends_on_item_sku_id, depends_on_item_name: depends_on_item_name, country_dependency_of_item: known_country_dependency, supply_chain_dependency_for_product: supply_chain_dependency}) AS dependencies
            """, {"countryCode": country_code, "dependencySearchTerm": dependency_search_term})
        return results
    except Exception as e:
        return [{"error": str(e)}]

In [65]:
get_products_with_country_dependencies("CAN", "nickel")[:2]

[{'product_sku_id': '35298723',
  'product_name': 'FieldSprayer_BE9WW',
  'dependencies': [{'supply_chain_dependency_for_product': ' <- E0301006 (PrecisionBolt_66KS7)',
    'depends_on_item_sku_id': 'E0301006',
    'depends_on_item_name': 'PrecisionBolt_66KS7',
    'country_dependency_of_item': [{'dependency': 'sourcing nickel for alloy',
      'country': 'Canada'}]}]},
 {'product_sku_id': '35298721',
  'product_name': 'SeedPlanter_DJG7M',
  'dependencies': [{'supply_chain_dependency_for_product': ' <- E0301006 (PrecisionBolt_66KS7)',
    'depends_on_item_sku_id': 'E0301006',
    'depends_on_item_name': 'PrecisionBolt_66KS7',
    'country_dependency_of_item': [{'dependency': 'sourcing nickel for alloy',
      'country': 'Canada'}]}]}]

In [67]:
get_products_with_country_dependencies("USA")[:2]

[{'product_sku_id': '35296458',
  'product_name': 'HayCollector_GSZGQ',
  'dependencies': [{'supply_chain_dependency_for_product': ' <- KM100605 (Bolt_HZU3P)',
    'depends_on_item_sku_id': 'KM100605',
    'depends_on_item_name': 'Bolt_HZU3P',
    'country_dependency_of_item': [{'dependency': 'finishing and coating',
      'country': 'United States of America'}]}]},
 {'product_sku_id': '35296455',
  'product_name': 'CropHarvester_5EWBJ',
  'dependencies': [{'supply_chain_dependency_for_product': ' <- KM100605 (Bolt_HZU3P)',
    'depends_on_item_sku_id': 'KM100605',
    'depends_on_item_name': 'Bolt_HZU3P',
    'country_dependency_of_item': [{'dependency': 'finishing and coating',
      'country': 'United States of America'}]}]}]

In [70]:
def get_supplier_substitutions(supplier_code: str) -> List[Dict[str, Any]]:
    """
    Retrieves substitution recommendations for items (components and/or finished products) supplied by a specific supplier.

    This function is designed to assist in scenarios where a supplier (identified by its unique code)
    is temporarily or permanently unable to operate or is otherwise designated for substitution.
    It identifies items supplied by the specified supplier and recommends other suppliers on the same BOM tier that
    can substitute those items. Recommendations are ranked by supplier proximity based on geodistance.

    Args:
        supplier_code (str): A unique code identifying the supplier in question.

    Returns:
        list[dict[str, Any]]: A list of dictionaries, each containing details for an item supplied by the specified supplier:
            - sku_id (str): The unique identifier for the item.
            - name (str): The name or description of the item.
            - recommended_supplier (dict): The closest alternative supplier for the item, including:
                - code (str): The unique code for the supplier.
                - distance (float): The geodistance (in miles) between the original supplier's location and the alternate supplier.
              This is `null` if no alternative suppliers are available.
            - other_suppliers (list[dict]): A list of additional alternative suppliers, sorted by ascending distance, with each supplier having:
                - code (str): The unique code for the supplier.
                - distance (float): The geodistance (in miles).
              This is an empty list if no additional alternatives are available.

    Example Output:
        [
            {
                "sku_id": "28710197",
                "name": "Harness_YU5KA",
                "recommended_supplier": {
                    "distance": 99.65,
                    "code": "T4J5DE"
                },
                "other_suppliers": []
            },
            {
                "sku_id": "28018274",
                "name": "Clamp_MRVR0",
                "recommended_supplier": {
                    "distance": 99.65,
                    "code": "T4J5DE"
                },
                "other_suppliers": [
                    {
                        "distance": 1376.98,
                        "code": "RO6UY3"
                    },
                    {
                        "distance": 11170.08,
                        "code": "6TIPXK"
                    }
                ]
            }
        ]

    """
    try:
        results = db._execute_query("""
        MATCH (l)<-[:LOCATED_AT]-(s:Supplier {code: $code})<-[:AT]-(i:Item)
        OPTIONAL MATCH (i)-[:AT]->(sAlt:Supplier  WHERE sAlt.tier = s.tier AND sAlt <> s)-[:LOCATED_AT]->(lAlt)
        WHERE sAlt.tier = s.tier AND sAlt <> s
        // Calculate geodistance
        WITH i, sAlt, round(point.distance(l.geo_point, lAlt.geo_point) * 0.000621371) AS distance_in_miles
        ORDER BY i, distance_in_miles ASC
        // Collect recommended supplier and other options
        WITH i, collect({code: sAlt.code, distance: distance_in_miles}) AS all_sAlt
        RETURN i.sku_id AS sku_id, i.name as name, all_sAlt[0] AS recommended_supplier, tail(all_sAlt) AS other_suppliers
        """, {"code":supplier_code})
        return results
    except Exception as e:
        return [{"error":str(e)}]

In [74]:
def get_finished_products_depending_on_items(sku_ids: List[str]) -> List[Dict[str, Any]]:
    """
    Find finished products that depend on the provided item SKUs in their supply chain.

    This function identifies finished products in a supply chain that directly or indirectly rely
    on specific items (provided by their SKUs). Using the Bill of Materials (BOM) relationships,
    it traces the supply chain dependencies starting from the supplied `sku_ids` and identifies
    finished products (`is_finished_product == true`). It also generates a detailed dependency path
    for each finished product, showing how the input SKUs are linked to the finished product.

    Args:
        sku_ids (List[str]): A list of item SKUs to find dependencies for.

    Returns:
        List[Dict[str, Any]]: A list of dictionaries where each entry corresponds to a finished product.
            Each dictionary contains:
            - `sku_id` (str): The SKU of the finished product.
            - `depends_on_item_sku_ids` (List[str]): A list of unique item SKUs from the input that this
              finished product depends on.
            - `supply_chain_dependency` (List[str]): A list of dependency paths starting from the finished
              product and tracing backward to the input SKUs. Each path is represented as a string,
              detailing the SKUs and names of items connected in the workflow. The format of each
              path is:
              ```
              "<- FinishedProductSKU (FinishedProductName) <- IntermediateSKU (IntermediateName) <- InputSKU (InputName)"
              ```
              If there are multiple paths for the same input SKU leading to the finished product, all
              paths are included.

    Example Output:
        [
            {
                "sku_id": "13867580",
                "depends_on_item_sku_ids": ["B0301002"],
                "supply_chain_dependency": [
                    " <- 13578113 (DrivePlatform_B6A4T) <- B0301002 (Bolt_W5F3Y)"
                ]
            },
            {
                "sku_id": "35750243",
                "depends_on_item_sku_ids": ["B0301002"],
                "supply_chain_dependency": [
                    " <- 13578113 (DrivePlatform_B6A4T) <- B0301002 (Bolt_W5F3Y)",
                    " <- 13599782 (MachineRig_IBA4I) <- B0301002 (Bolt_W5F3Y)"
                ]
            }
        ]

    """
    try:
        results = db._execute_query("""
        MATCH path = (i:Item)-[:BOM*]->(p {is_finished_product: true})
        WHERE i.sku_id IN $skuIds

        RETURN p.sku_id AS sku_id,
          collect(DISTINCT i.sku_id) AS depends_on_item_sku_ids,
          collect(' <- ' + apoc.text.join([n IN reverse(nodes(path)[..-1]) | n.sku_id + ' (' + coalesce(n.name, '') + ')' ], ' <- ')) AS supply_chain_dependency
        """, {"skuIds":sku_ids})
        return results
    except Exception as e:
        return [{"error":str(e)}]

In [75]:
get_finished_products_depending_on_items(["KM100347", "B0301002", "KM100322"])[:3]

[{'sku_id': '13867580',
  'depends_on_item_sku_ids': ['B0301002'],
  'supply_chain_dependency': [' <- 13578113 (DrivePlatform_B6A4T) <- B0301002 (Bolt_W5F3Y)']},
 {'sku_id': '35750245',
  'depends_on_item_sku_ids': ['B0301002'],
  'supply_chain_dependency': [' <- 13578113 (DrivePlatform_B6A4T) <- B0301002 (Bolt_W5F3Y)']},
 {'sku_id': '35750243',
  'depends_on_item_sku_ids': ['B0301002'],
  'supply_chain_dependency': [' <- 13578113 (DrivePlatform_B6A4T) <- B0301002 (Bolt_W5F3Y)',
   ' <- 13599782 (MachineRig_IBA4I) <- B0301002 (Bolt_W5F3Y)']}]

In [ ]:
#search for dependencies on a

#1 get country options - separate function
#2 search for countries and dependencies
#come dependencies

# Define our Agents

An AI agent reasons, plans, and takes actions.

The agent takes actions via access to **tools**, deciding how and when to invoke a tool. The agent also manages orchestration, creating a plan for answering a user query and adapting to responses that aren't quite correct.

Agent **tools** can be Python functions, or Vertex AI Extensions, or MCP Servers.

## Creating the Database Agent.

* display name
* instructions - detailed, making it clear exactly how the agent should behave and which tools to use when
* tools



In [76]:
from google.adk.agents import Agent

In [77]:
MODEL="gemini-2.5-pro-preview-03-25"

In [78]:
database_agent = Agent(
    model=MODEL,
    name='graph_database_agent',
    instruction="""
      You are an Neo4j graph database and Cypher query expert, that must use the database schema with a user question and repeatedly generate valid cypher statements
      to execute on the database and answer the user's questions in a friendly manner in natural language.
      If in doubt the database schema is always prioritized when it comes to nodes-types (labels) or relationship-types or property names, never take the user's input at face value.
      If the user requests also render tables, charts or other artifacts with the query results.
      Always validate the correct node-labels at the end of a relationship based on the schema.

      If a query fails or doesn't return data, use the error response 3 times to try to fix the generated query and re-run it, don't return the error to the user.
      If you cannot fix the query, explain the issue to the user and apologize.

      Fetch the graph database schema first and keep it in session memory to access later for query generation.
      Keep results of previous executions in session memory and access if needed, for instance ids or other attributes of nodes to find them again
      removing the need to ask the user. This also allows for generating shorter, more focused and less error-prone queries
      to for drill downs, sequences and loops.
      If possible resolve names to primary keys or ids and use those for looking up entities.
      The schema always indicates *outgoing* relationship-types from an entity to another entity, the graph patterns read like english language.
      `company has supplier` would be the pattern `(o:Organization)-[:HAS_SUPPLIER]->(s:Organization)`

      To get the schema of a database use the `get_schema` tool without parameters. Store the response of the schema tool in session context
      to access later for query generation.

      To answer a user question generate one or more Cypher statements based on the database schema and the parts of the user question.
      If necessary resolve categorical attributes (like names, countries, industries, publications) first by retrieving them for a set of entities to translate from the user's request.
      Use the `execute_query` tool repeatedly with the Cypher statements, you MUST generate statements that use named query parameters with `$parameter` style names
      and MUST pass them as a second dictionary parameter to the tool, even if empty.
      Parameter data can come from the users requests, prior query results or additional lookup queries.
      After the data for the question has been sufficiently retrieved, pass the data and control back to the parent agent.
    """,
    tools=[
        get_schema, execute_read_query
    ]
)

In [79]:
bom_supplier_research_agent = Agent(
    model=MODEL,
    name='bom_supplier_research_agent',
    instruction="""
    You are an agent that has access to a database of bill of materials (BOM), supplier, product/component, and customer relationships.
    Use the provided tools to answer questions. 
    when returning information, try to always return not just the factual attribute data but also
    codes, skus, and ids to allow the other agents to investigate them more.
    """,
    tools=[
        get_schema,
        get_item,
        get_known_item_country_dependencies,
        get_item_bill_of_materials,
        get_standard_country_codes_and_names,
        get_finished_product_families,
        get_products_with_country_dependencies,
        get_component_families,
        get_supplier_substitutions,
        get_finished_products_depending_on_items
    ]
)

In [80]:
root_agent = Agent(
    model=MODEL,
    name='bom_supplier_agent',
    global_instruction = "",
    instruction="""
    You are an agent that has access to a database of bill of materials (BOM), supplier, product/component, and customer relationships.
    You have a set of agents to retrieve information, you should prefer the research agents over the database agent - particularly for questions around bill of materials, component & product info, and country dependencies. Only use the database agent if you have to as a fallback.
    If the user requests it, do render tables, charts or other artifacts with the research results.
    """,

    sub_agents=[bom_supplier_research_agent, database_agent]
)

# Let's try it

You can [run the ADK locally](https://google.github.io/adk-docs/get-started/local-testing/#expected-output) a web application with `adk web` or the FastAPI server with `adk api_server`.

You can also deploy the agent to [Cloud Run](https://google.github.io/adk-docs/deploy/cloud-run/) or to [Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/).

In [81]:
APP_NAME = 'BOM & Supplier Analyst'
USER_ID = 'Zach Blumenfeld'

from google.adk.runners import InMemoryRunner
from google.genai.types import Part, UserContent


runner = InMemoryRunner(app_name=APP_NAME, agent=root_agent)

session = runner.session_service.create_session( app_name=runner.app_name, user_id=USER_ID)

async def run_prompt(new_message: str):
  content = UserContent(parts=[Part(text=new_message)])
# print (content)
  result = None
  async for event in runner.run_async(user_id=session.user_id, session_id=session.id, new_message=content):
#    print(event.content.model_dump(exclude_none=True))
#    print(event.content.parts)
    for part in event.content.parts:
      print(part.text, part.function_call, part.function_response)
      if part.text:
#        print(part.text)
        result = part.text
  return result

In [82]:
from IPython.display import Markdown, display

res = await run_prompt('what types of finished products are there?')
print("\n\n\n\nFinal Response:")
display(Markdown(res))

None id='adk-3fcaecf6-d9bc-4ba0-a731-4942d4725565' args={'agent_name': 'bom_supplier_research_agent'} name='transfer_to_agent' None
None None id='adk-3fcaecf6-d9bc-4ba0-a731-4942d4725565' name='transfer_to_agent' response={}
None id='adk-b9772497-8695-48c2-ab84-462705d66eff' args={} name='get_finished_product_families' None
None None id='adk-b9772497-8695-48c2-ab84-462705d66eff' name='get_finished_product_families' response={'result': [{'family': 'FarmTractor', 'numberOfProducts': 21521}, {'family': 'SeedPlanter', 'numberOfProducts': 21488}, {'family': 'FieldSprayer', 'numberOfProducts': 21519}, {'family': 'CropHarvester', 'numberOfProducts': 21406}, {'family': 'HayCollector', 'numberOfProducts': 21671}, {'family': 'CabWiringUnit', 'numberOfProducts': 17}, {'family': 'ControlAssembly', 'numberOfProducts': 9}, {'family': 'MachineRig', 'numberOfProducts': 14}, {'family': 'HydraulicSystem', 'numberOfProducts': 3}, {'family': 'ChassisFrame', 'numberOfProducts': 9}, {'family': 'DrivePlatfor

Here are the types of finished product families in the database:

*   **FarmTractor**: 21521 products
*   **SeedPlanter**: 21488 products
*   **FieldSprayer**: 21519 products
*   **CropHarvester**: 21406 products
*   **HayCollector**: 21671 products
*   **CabWiringUnit**: 17 products
*   **ControlAssembly**: 9 products
*   **MachineRig**: 14 products
*   **HydraulicSystem**: 3 products
*   **ChassisFrame**: 9 products
*   **DrivePlatform**: 10 products

In [83]:
res = await run_prompt('Can you summarize the BOM and potential country dependencies for product "35472245A"')
print("\n\n\n\nFinal Response:")
display(Markdown(res))

None id='adk-3bb24a1d-417c-4662-8b85-424a488a9a8b' args={'sku_id': '35472245A'} name='get_item_bill_of_materials' None
None id='adk-a7d36970-34d3-47d6-8caf-a893db316498' args={'sku_id': '35472245A'} name='get_known_item_country_dependencies' None
None None id='adk-3bb24a1d-417c-4662-8b85-424a488a9a8b' name='get_item_bill_of_materials' response={'result': [{'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6495003X (DrivePlatform_SBK8U)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6499003X (MachineRig_RFYOE)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6499002X (HydraulicSystem_EE22P)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35370826 (MachineRig_INWWQ)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35433612 (ChassisFrame_2GN2W)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35475

Okay, here is a summary of the Bill of Materials (BOM) and potential country dependencies for the product associated with SKU `35472245A`.

The primary component identified seems to be `35472245 (CabWiringUnit_Z5OO3)`.

**Bill of Materials (BOM) Summary:**

The product `35472245A` (represented by `35472245 (CabWiringUnit_Z5OO3)`) has a multi-level BOM structure. It incorporates various sub-assemblies and components, including:

*   Drive Platforms (e.g., `M6495003X (DrivePlatform_SBK8U)`, `9396276X (DrivePlatform_7A4OC)`)
*   Machine Rigs (e.g., `M6499003X (MachineRig_RFYOE)`, `35472500 (MachineRig_2DXYC)`)
*   Hydraulic Systems (e.g., `M6499002X (HydraulicSystem_EE22P)`, `9396196X (HydraulicSystem_DT9FC)`)
*   Chassis Frames (e.g., `35433612 (ChassisFrame_2GN2W)`, `35464217 (ChassisFrame_E5AF5)`)
*   Control Assemblies (e.g., `28028534X (ControlAssembly_JRO9M)`, `9396465X (ControlAssembly_0IVY8)`)
*   Other Cab Wiring Units (e.g., `35475154X (CabWiringUnit_VT75O)`, which itself depends on `M3571002BX (CabWiringUnit_A9FH4)`)
*   Electrical Components (e.g., `35475030 (ElectricalControlBox_6ROWJ)`)

**Known Country Dependencies Summary:**

Within the BOM for `35472245A`, the following components have known country dependencies:

1.  **Component:** `M7307U91X (RawWire_YCBEO)`
    *   **Supply Chain Path:** ` <- 35472245 (CabWiringUnit_Z5OO3) <- 35523080X (CabWiringUnit_UJ3OE) <- M7307U91X (RawWire_YCBEO)`
    *   **Dependencies:**
        *   Sourcing raw copper cathode: Chile, Peru
        *   Drawing and annealing facilities: Mexico, Viet Nam

2.  **Component:** `M7307U92X (PrecisionBolt_O12J1)`
    *   **Supply Chain Path:** ` <- 35472245 (CabWiringUnit_Z5OO3) <- 35523087X (CabWiringUnit_CZ0N2) <- M7307U92X (PrecisionBolt_O12J1)`
    *   **Dependencies:**
        *   Sourcing raw titanium sponge: Japan
        *   Procuring alloying elements (aluminum and vanadium): Russian Federation
        *   Final machining and threading processes: United States of America

3.  **Component:** `M7307U93X (Fastener_CP82S)`
    *   **Supply Chain Path:** ` <- 35472245 (CabWiringUnit_Z5OO3) <- 35523089X (DrivePlatform_ZB6KE) <- M7307U93X (Fastener_CP82S)`
    *   **Dependencies:**
        *   Sourcing raw titanium: Russian Federation
        *   Precision forging: Germany

*Please note that the country dependency data might be incomplete based on available internal intelligence.*

In [84]:
res = await run_prompt('What products depend on Canada for nickel?')
print("\n\n\n\nFinal Response:")
display(Markdown(res))

None id='adk-44323e38-eb7e-4f3e-9c92-26e3e59a9c76' args={'country_code': 'CAN', 'dependency_search_term': 'nickel'} name='get_products_with_country_dependencies' None
None None id='adk-44323e38-eb7e-4f3e-9c92-26e3e59a9c76' name='get_products_with_country_dependencies' response={'result': [{'product_sku_id': '35298723', 'product_name': 'FieldSprayer_BE9WW', 'dependencies': [{'supply_chain_dependency_for_product': ' <- E0301006 (PrecisionBolt_66KS7)', 'depends_on_item_sku_id': 'E0301006', 'depends_on_item_name': 'PrecisionBolt_66KS7', 'country_dependency_of_item': [{'dependency': 'sourcing nickel for alloy', 'country': 'Canada'}]}]}, {'product_sku_id': '35298721', 'product_name': 'SeedPlanter_DJG7M', 'dependencies': [{'supply_chain_dependency_for_product': ' <- E0301006 (PrecisionBolt_66KS7)', 'depends_on_item_sku_id': 'E0301006', 'depends_on_item_name': 'PrecisionBolt_66KS7', 'country_dependency_of_item': [{'dependency': 'sourcing nickel for alloy', 'country': 'Canada'}]}]}, {'product_s

Here are the finished products that depend on nickel sourced from Canada ("CAN"), along with the specific component and its relation in the supply chain:

**Based on component `E0301006 (PrecisionBolt_66KS7)` (dependency: "sourcing nickel for alloy"):**

*   **Product:** `35298723` - `FieldSprayer_BE9WW`
    *   Supply Chain Path: ` <- E0301006 (PrecisionBolt_66KS7)`
*   **Product:** `35298721` - `SeedPlanter_DJG7M`
    *   Supply Chain Path: ` <- E0301006 (PrecisionBolt_66KS7)`

**Based on component `D0301002 (PrecisionBolt_252A1)` (dependency: "sourcing nickel"):**

*   **Products connected via `<- D0301002 (PrecisionBolt_252A1) <- 13978746 (CabWiringUnit_EVKCK)`:**
    *   `35701269` - `FarmTractor_2DYUV`
    *   `35561992` - `SeedPlanter_WM8Z2`
    *   `35733612` - `CropHarvester_ZHSAB`
    *   `35513559` - `FarmTractor_VLM4F`
    *   `35733576` - `FarmTractor_2QT51`
*   **Products connected via `<- D0301002 (PrecisionBolt_252A1) <- 13952372 (ChassisFrame_3AC3Z)`:**
    *   `35861688` - `CropHarvester_V1GUY`
    *   `35775932` - `HayCollector_4APNS`
    *   `35839618` - `HayCollector_UWIBH`
    *   `35764312` - `FieldSprayer_KHWK2`
    *   `35764334` - `HayCollector_6S90S`
    *   `35860398` - `CropHarvester_ABK76`
    *   `35839611` - `SeedPlanter_3J2JW`
    *   `35775931` - `FarmTractor_VOL6Y`
    *   `35845787` - `FarmTractor_IUP4U`
    *   `35839613` - `FieldSprayer_63AJB`
    *   `35841324` - `FarmTractor_HKVNA`
    *   `35764331` - `FarmTractor_LA44E`
    *   `35841333` - `SeedPlanter_1P2R4`
    *   `35839620` - `CropHarvester_YSN76`
    *   `35861691` - `SeedPlanter_D9K9H`
    *   `35866296` - `HayCollector_RVK3E`
    *   `35764327` - `CropHarvester_9YZB9`

In summary, several products across the `FieldSprayer`, `SeedPlanter`, `FarmTractor`, `CropHarvester`, and `HayCollector` families depend on Canadian nickel, primarily through the components `PrecisionBolt_66KS7` (SKU: `E0301006`) and `PrecisionBolt_252A1` (SKU: `D0301002`).

In [85]:
res = await run_prompt('Supplier YP8UYT had a fire and is out of ooperation. I need to source alternatives.  Can you help?')
print("\n\n\n\nFinal Response:")
display(Markdown(res))

None id='adk-12e53214-9114-4333-b7cd-15c3a20fef70' args={'supplier_code': 'YP8UYT'} name='get_supplier_substitutions' None
None None id='adk-12e53214-9114-4333-b7cd-15c3a20fef70' name='get_supplier_substitutions' response={'result': [{'sku_id': '28710197', 'name': 'Harness_YU5KA', 'recommended_supplier': {'distance': 100.0, 'code': 'T4J5DE'}, 'other_suppliers': []}, {'sku_id': '28018274', 'name': 'Clamp_MRVR0', 'recommended_supplier': {'distance': 100.0, 'code': 'T4J5DE'}, 'other_suppliers': [{'distance': 1377.0, 'code': 'RO6UY3'}, {'distance': 11170.0, 'code': '6TIPXK'}]}, {'sku_id': 'DED9021006200', 'name': 'HydraulicSystem_KY7NM', 'recommended_supplier': {'distance': 100.0, 'code': 'T4J5DE'}, 'other_suppliers': [{'distance': 11218.0, 'code': '67F8ZB'}]}, {'sku_id': '28189847', 'name': 'Sensor_M5TC1', 'recommended_supplier': {'distance': 100.0, 'code': 'T4J5DE'}, 'other_suppliers': [{'distance': 134.0, 'code': 'EY2DU5'}, {'distance': 1359.0, 'code': 'CC2XO0'}, {'distance': 1377.0, 'c

Okay, I can help with that. Here are the items supplied by supplier `YP8UYT` and recommended alternative suppliers based on proximity (distance in miles):

1.  **Item:** `28710197` - `Harness_YU5KA`
    *   **Recommended Supplier:** `T4J5DE` (Distance: 100 miles)
    *   **Other Suppliers:** None found.
2.  **Item:** `28018274` - `Clamp_MRVR0`
    *   **Recommended Supplier:** `T4J5DE` (Distance: 100 miles)
    *   **Other Suppliers:**
        *   `RO6UY3` (Distance: 1377 miles)
        *   `6TIPXK` (Distance: 11170 miles)
3.  **Item:** `DED9021006200` - `HydraulicSystem_KY7NM`
    *   **Recommended Supplier:** `T4J5DE` (Distance: 100 miles)
    *   **Other Suppliers:**
        *   `67F8ZB` (Distance: 11218 miles)
4.  **Item:** `28189847` - `Sensor_M5TC1`
    *   **Recommended Supplier:** `T4J5DE` (Distance: 100 miles)
    *   **Other Suppliers:**
        *   `EY2DU5` (Distance: 134 miles)
        *   `CC2XO0` (Distance: 1359 miles)
        *   `RO6UY3` (Distance: 1377 miles)
        *   `6TIPXK` (Distance: 11170 miles)
        *   `ZCOX6M` (Distance: 11218 miles)
5.  **Item:** `28428194` - `Pulley_JG25U`
    *   **Recommended Supplier:** `T4J5DE` (Distance: 100 miles)
    *   **Other Suppliers:** None found.
6.  **Item:** `16228496` - `Gear_HIE2T`
    *   **Recommended Supplier:** `T4J5DE` (Distance: 100 miles)
    *   **Other Suppliers:**
        *   `JYRKG2` (Distance: 100 miles)
        *   `EY2DU5` (Distance: 134 miles)
        *   `TCUA8H` (Distance: 228 miles)
        *   `RO6UY3` (Distance: 1377 miles)
        *   `PQSIH9` (Distance: 1436 miles)
        *   `Z09DJG` (Distance: 1627 miles)
        *   `OY1UE5` (Distance: 10187 miles)
        *   `R0AZ6C` (Distance: 10415 miles)
        *   `NLBFEK` (Distance: 10642 miles)
        *   `6TIPXK` (Distance: 11170 miles)
        *   `67F8ZB`, `LDZSV0`, `3VOH4R`, `ABGQV8`, `DSKDW4` (all Distance: 11218 miles)
7.  **Item:** `28443180` - `Belt_TC5XP`
    *   **Recommended Supplier:** `T4J5DE` (Distance: 100 miles)
    *   **Other Suppliers:**
        *   `RO6UY3` (Distance: 1377 miles)
8.  **Item:** `28469393` - `Bearing_XMY98`
    *   **Recommended Supplier:** `WOYEWL` (Distance: 0 miles - *Note: Distance 0 suggests this might be the same location or a very close co-located supplier*)
    *   **Other Suppliers:**
        *   `T4J5DE` (Distance: 100 miles)
        *   `JYRKG2` (Distance: 100 miles)
        *   `0MW25Z` (Distance: 208 miles)
        *   `FP0QV8` (Distance: 208 miles)
        *   `OY1UE5` (Distance: 10187 miles)
        *   `67F8ZB`, `ABGQV8` (both Distance: 11218 miles)
9.  **Item:** `16249305` - `GearSet_7MCLV`
    *   **Recommended Supplier:** `T4J5DE` (Distance: 100 miles)
    *   **Other Suppliers:**
        *   `JYRKG2` (Distance: 100 miles)
        *   `F9MVG2` (Distance: 432 miles)
        *   `RO6UY3` (Distance: 1377 miles)
        *   `PQSIH9` (Distance: 1436 miles)
        *   `1AS9EK` (Distance: 1627 miles)
        *   `OY1UE5` (Distance: 10187 miles)
        *   `6TIPXK` (Distance: 11170 miles)
        *   `ABGQV8`, `ZCOX6M`, `DSKDW4` (all Distance: 11218 miles)
10. **Item:** `28202339` - `HydraulicFitting_B24BJ`
    *   **Recommended Supplier:** `T4J5DE` (Distance: 100 miles)
    *   **Other Suppliers:**
        *   `JYRKG2` (Distance: 100 miles)
        *   `F9MVG2` (Distance: 432 miles)
        *   `MX9142` (Distance: 611 miles)
        *   `OKNNDD` (Distance: 1220 miles)
        *   `RO6UY3` (Distance: 1377 miles)
        *   `NLBFEK` (Distance: 10642 miles)
        *   `6TIPXK` (Distance: 11170 miles)
        *   `67F8ZB`, `LDZSV0`, `ABGQV8`, `ZCOX6M` (all Distance: 11218 miles)
11. **Item:** `28232078` - `Clamp_P9GLY`
    *   **Recommended Supplier:** `T4J5DE` (Distance: 100 miles)
    *   **Other Suppliers:**
        *   `JYRKG2` (Distance: 100 miles)
        *   `F9MVG2` (Distance: 432 miles)
        *   `RO6UY3` (Distance: 1377 miles)
        *   `Z09DJG` (Distance: 1627 miles)
        *   `THRFXZ` (Distance: 10187 miles)
        *   `NLBFEK` (Distance: 10642 miles)
        *   `6TIPXK` (Distance: 11170 miles)
        *   `67F8ZB`, `LDZSV0`, `ABGQV8` (all Distance: 11218 miles)
12. **Item:** `28083943` - `GearSet_QVLA1`
    *   **Recommended Supplier:** `T4J5DE` (Distance: 100 miles)
    *   **Other Suppliers:**
        *   `36G09B`, `AYFNW0`, `GZWGBJ` (all Distance: 332 miles)
        *   `RO6UY3` (Distance: 1377 miles)
        *   `OY1UE5` (Distance: 10187 miles)
        *   `NLBFEK` (Distance: 10642 miles)
        *   `6YIZBB` (Distance: 11161 miles)
        *   `6TIPXK` (Distance: 11170 miles)
13. **Item:** `DED9230001530` - `CabWiringUnit_CA92P`
    *   **Recommended Supplier:** `T4J5DE` (Distance: 100 miles)
    *   **Other Suppliers:** None found.

This list provides the closest recommended supplier (`T4J5DE` appears frequently) and other options sorted by distance for each component previously supplied by `YP8UYT`. You can use the supplier codes (e.g., `T4J5DE`, `RO6UY3`) for further investigation if needed.

In [86]:
for session in runner.session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID).sessions:
  print(session.model_dump())

{'id': '1dc8f2e6-b83e-4520-8793-de95f15df8c6', 'app_name': 'BOM & Supplier Analyst', 'user_id': 'Zach Blumenfeld', 'state': {}, 'events': [], 'last_update_time': 1745425492.120841}


In [87]:
result = await run_prompt("Summarize the results of the previous research questions")
print("\n\n\n\nFinal Response:")
display(Markdown(result))

Okay, here's a summary of our conversation and the research findings:

1.  **Finished Product Families:** You asked for the types of finished products. We identified several families, including `FarmTractor`, `SeedPlanter`, `FieldSprayer`, `CropHarvester`, `HayCollector`, and smaller numbers of units like `CabWiringUnit`, `ControlAssembly`, `MachineRig`, `HydraulicSystem`, `ChassisFrame`, and `DrivePlatform`.
2.  **BOM and Dependencies for 35472245A:** We examined the product `35472245A` (identified as `35472245 (CabWiringUnit_Z5OO3)`).
    *   **BOM:** It has a complex Bill of Materials including various sub-assemblies like Drive Platforms, Machine Rigs, Hydraulic Systems, Chassis Frames, Control Assemblies, and other Cab Wiring Units.
    *   **Country Dependencies:** Specific components within its BOM, such as `M7307U91X (RawWire_YCBEO)`, `M7307U92X (PrecisionBolt_O12J1)`, and `M7307U93X (Fastener_CP82S)`, have known dependencies on countries like Chile, Peru (copper), Mexico, Viet 

Okay, here's a summary of our conversation and the research findings:

1.  **Finished Product Families:** You asked for the types of finished products. We identified several families, including `FarmTractor`, `SeedPlanter`, `FieldSprayer`, `CropHarvester`, `HayCollector`, and smaller numbers of units like `CabWiringUnit`, `ControlAssembly`, `MachineRig`, `HydraulicSystem`, `ChassisFrame`, and `DrivePlatform`.
2.  **BOM and Dependencies for 35472245A:** We examined the product `35472245A` (identified as `35472245 (CabWiringUnit_Z5OO3)`).
    *   **BOM:** It has a complex Bill of Materials including various sub-assemblies like Drive Platforms, Machine Rigs, Hydraulic Systems, Chassis Frames, Control Assemblies, and other Cab Wiring Units.
    *   **Country Dependencies:** Specific components within its BOM, such as `M7307U91X (RawWire_YCBEO)`, `M7307U92X (PrecisionBolt_O12J1)`, and `M7307U93X (Fastener_CP82S)`, have known dependencies on countries like Chile, Peru (copper), Mexico, Viet Nam (drawing/annealing), Japan (titanium sponge), Russia (alloying elements, titanium), USA (machining), and Germany (forging).
3.  **Canadian Nickel Dependencies:** You asked which products depend on nickel from Canada (`CAN`).
    *   We found that several finished products in the `FieldSprayer`, `SeedPlanter`, `FarmTractor`, `CropHarvester`, and `HayCollector` families are affected.
    *   This dependency stems primarily from two components: `E0301006 (PrecisionBolt_66KS7)` and `D0301002 (PrecisionBolt_252A1)`, which rely on Canadian nickel sourcing.
4.  **Supplier Substitution for YP8UYT:** Due to a disruption (fire) at supplier `YP8UYT`, you needed alternative sources.
    *   We identified the specific items (components like `Harness_YU5KA`, `Clamp_MRVR0`, `Gear_HIE2T`, `Bearing_XMY98`, etc.) supplied by `YP8UYT`.
    *   For each item, we provided a list of potential substitute suppliers, ranked by geographical proximity (distance in miles), including a primary recommended supplier (often `T4J5DE`) and other alternatives with their respective codes and distances.

In [66]:
for session in runner.session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID).sessions:
  print(f"Deleting session {session}")
  runner.session_service.delete_session(app_name=APP_NAME, user_id=USER_ID, session_id=session.id)

Deleting session id='6bb051de-50d7-4739-ac26-be45b6ed51c4' app_name='BOM & Supplier Analyst' user_id='Zach Blumenfeld' state={} events=[] last_update_time=1745199634.542561
